In [ ]:

import sys
import numpy as np
from hmmlearn.hmm import CategoricalHMM

# 
STATE_NAMES = ["不喜歡", "有好感", "喜歡"]
OBS_NAMES = {1: "愉快地互動", 2: "友善回應", 3: "冷漠"}

# 建立 HMM 模型
def build_model():
    # 
    n_states, n_obs = 3, 3
    model = CategoricalHMM(n_components=n_states, n_features=n_obs, init_params="")

    model.startprob_ = np.array([0.20, 0.60, 0.20])

    model.transmat_ = np.array([
        [0.72, 0.23, 0.05],
        [0.12, 0.68, 0.20],
        [0.05, 0.22, 0.73],
    ])
    model.emissionprob_ = np.array([
        [0.05, 0.18, 0.77],
        [0.25, 0.60, 0.15],
        [0.72, 0.24, 0.04],
    ])
    return model


def story_line(state_idx: int, obs: int, step: int):
    """
    小劇場：用 (隱狀態, 觀測) 組合生成一句描述（固定模板，不依賴 random，方便可重現）
    """
    s = STATE_NAMES[state_idx]
    o = OBS_NAMES[obs]

    # 讓敘事有點「推理感」：同一觀測在不同情緒下有不同解釋
    lines = {
        ("不喜歡", 1): [
            "她笑得很禮貌，但眼神像是在把時間切成一格一格地消耗。",
            "她的愉快像是職業反射：聲音上揚，內容卻不接球。"
        ],
        ("不喜歡", 2): [
            "她回得得體，像把話題放在托盤上遞回來，沒有多一點溫度。",
            "她很友善，但友善不等於靠近；你感覺她在維持安全距離。"
        ],
        ("不喜歡", 3): [
            "她看向窗外的時間比看向你的時間長，冷漠是一種明確訊號。",
            "她的回應短、慢、乾淨俐落，像是在結束一段不必要的對話。"
        ],
        ("有好感", 1): [
            "她主動把話題接起來，還順手丟回一顆更有趣的球。",
            "她笑出聲的瞬間很自然，像是你剛好踩到她的笑點。"
        ],
        ("有好感", 2): [
            "她的友善帶著延伸：會追問細節，也會分享自己的版本。",
            "她點頭的節奏跟你說話同步，像在說：我有在聽。"
        ],
        ("有好感", 3): [
            "她短暫冷了一下，可能是被某句話戳到雷點，也可能只是累。",
            "她沉默了一拍，但沒有逃走；更像在重新評估你這個人。"
        ],
        ("喜歡", 1): [
            "她的眼睛亮了一下，笑意不是禮貌，是失手洩漏的真心。",
            "她不只接球，還加速：把聊天變成一場雙人默契遊戲。"
        ],
        ("喜歡", 2): [
            "她很友善，而且那種友善帶著偏心：會特別記你剛剛講的小細節。",
            "她的語氣柔軟，像是已經把你放進『可以信任』的名單。"
        ],
        ("喜歡", 3): [
            "她忽然冷了一下，但不像拒絕，更像在掩飾過於明顯的在意。",
            "她把情緒收起來，可能是害羞或擔心太快被你看穿。"
        ],
    }

    key = (s, obs)
    pool = lines.get(key, [f"她呈現「{o}」，你試著推測她此刻是「{s}」。"])
    # 用 step 來選句子，確保 deterministic
    return pool[step % len(pool)]


def pretty_probs(p):
    # p: shape (3,)
    items = list(zip(STATE_NAMES, p.tolist()))
    items.sort(key=lambda x: x[1], reverse=True)
    return " / ".join([f"{name} {prob:.3f}" for name, prob in items])



if __name__ == "__main__":
    model = build_model()

    hint = '''\
咖啡廳 HMM 推理器
- 隱藏狀態(女生真實情緒): 0不喜歡 / 1有好感 / 2喜歡
- 觀測值 (你看到的反應，請用 1/2/3 輸入):
    1 = 愉快地互動
    2 = 友善回應
    3 = 冷漠
輸入範例：1 2 2 3 2 1
離開：直接按 Enter
'''
    print(hint)

    while True:
        s = input("請輸入觀測序列(1/2/3): ").strip()

        if not s:
            print("結束。")
            break

        obs_list = [int(num )for num in s.split(" ")]

        X = np.array([o - 1 for o in obs_list], dtype=int).reshape(-1, 1)

        # Viterbi：最可能的隱狀態序列
        logprob, states = model.decode(X, algorithm="viterbi")

        # Posterior：每一步各狀態的機率
        post = model.predict_proba(X)  # shape (T, 3)

        print("\n" + "-" * 72)
        print(f"你的觀測：{obs_list}  ->  {[OBS_NAMES[o] for o in obs_list]}")
        print(f"最可能心情序列：{states.tolist()}  ->  {[STATE_NAMES[i] for i in states]}")
        print(f"路徑對數似然 (log P): {logprob:.3f}")
        print("-" * 72)

        # 逐步輸出推理 + 小劇場
        for t, (o, st) in enumerate(zip(obs_list, states), start=1):
            print(f"[第{t:02d}步] 觀測={o}({OBS_NAMES[o]}) | 推測心情={st}({STATE_NAMES[st]})")
            print(f"        後驗機率: {pretty_probs(post[t-1])}")
            print(f"        小劇場: {story_line(st, o, t)}")

        # 最後一步摘要（更接近你「想知道她到底是什麼情緒」）
        last_p = post[-1]
        best = int(np.argmax(last_p))
        confidence = float(last_p[best])

        print("-" * 72)
        print(f"此刻(最後一步)最可能情緒：{STATE_NAMES[best]}  (後驗={confidence:.3f})")
        if confidence >= 0.80:
            print("判讀：訊號相當明確。")
        elif confidence >= 0.60:
            print("判讀：偏向明顯，但仍可能被情境雜訊影響。")
        else:
            print("判讀：不確定性高，建議拉長觀測序列或提高觀測品質（更細的反應類別）。")
        print("-" * 72 + "\n")

咖啡廳 HMM 推理器
- 隱藏狀態(女生真實情緒): 0不喜歡 / 1有好感 / 2喜歡
- 觀測值 (你看到的反應，請用 1/2/3 輸入):
    1 = 愉快地互動
    2 = 友善回應
    3 = 冷漠
輸入範例：1 2 2 3 2 1
離開：直接按 Enter


------------------------------------------------------------------------
你的觀測：[1, 2, 2, 1, 1, 1, 1]  ->  ['愉快地互動', '友善回應', '友善回應', '愉快地互動', '愉快地互動', '愉快地互動', '愉快地互動']
最可能心情序列：[1, 1, 1, 2, 2, 2, 2]  ->  ['有好感', '有好感', '有好感', '喜歡', '喜歡', '喜歡', '喜歡']
路徑對數似然 (log P): -7.558
------------------------------------------------------------------------
[第01步] 觀測=1(愉快地互動) | 推測心情=1(有好感)
        後驗機率: 有好感 0.589 / 喜歡 0.394 / 不喜歡 0.017
        小劇場: 她笑出聲的瞬間很自然，像是你剛好踩到她的笑點。
[第02步] 觀測=2(友善回應) | 推測心情=1(有好感)
        後驗機率: 有好感 0.684 / 喜歡 0.297 / 不喜歡 0.019
        小劇場: 她的友善帶著延伸：會追問細節，也會分享自己的版本。
[第03步] 觀測=2(友善回應) | 推測心情=1(有好感)
        後驗機率: 有好感 0.598 / 喜歡 0.389 / 不喜歡 0.013
        小劇場: 她點頭的節奏跟你說話同步，像在說：我有在聽。
[第04步] 觀測=1(愉快地互動) | 推測心情=2(喜歡)
        後驗機率: 喜歡 0.787 / 有好感 0.210 / 不喜歡 0.003
        小劇場: 她的眼睛亮了一下，笑意不是禮貌，是失手洩漏的真心。
[第05步] 觀測=1(愉快地互動) | 推測心情=2(喜歡)
        後驗機率